In [1]:
import GetOldTweets3 as got
import pandas as pd
import requests
from bs4 import BeautifulSoup
import gensim

In [61]:
tweetCriteria = got.manager.TweetCriteria().setNear("Denver, Co")\
                                            .setWithin('15 miles')\
                                            .setMaxTweets(10)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)



Those are all the breweries within 15 miles of me right now. #holyshit #brewerymap #amazing… https://www.instagram.com/p/BAz3kvwPG7A/


In [2]:
tweetCriteria = got.manager.TweetCriteria().setNear("Paradise, Ca")\
                                            .setSince("2018-11-07")\
                                           .setUntil("2018-11-15")\
                                           .setMaxTweets(30)
tweets = got.manager.TweetManager.getTweets(tweetCriteria)


In [3]:
def get_twitter_info():
    tweet_df["id"] = tweet_df["got_criteria"].apply(lambda x: x.id)
    tweet_df["tweet_text"] = tweet_df["got_criteria"].apply(lambda x: x.text)
    tweet_df["date"] = tweet_df["got_criteria"].apply(lambda x: x.date)
    tweet_df["hashtags"] = tweet_df["got_criteria"].apply(lambda x: x.hashtags)
    tweet_df["username"] = tweet_df["got_criteria"].apply(lambda x: x.username)
    tweet_df["mentions"] = tweet_df["got_criteria"].apply(lambda x: x.mentions)
    tweet_df["rewtweets"] = tweet_df["got_criteria"].apply(lambda x: x.retweets)
    tweet_df["replies"] = tweet_df["got_criteria"].apply(lambda x: x.replies)
    tweet_df["link"] = tweet_df["got_criteria"].apply(lambda x: x.permalink)
# https://medium.com/@robbiegeoghegan/download-twitter-data-with-10-lines-of-code-42eb2ba1ab0f
#set search variables
keyword = "fire"
oldest_date = "2018-11-01"    
newest_date = "2018-11-15"
locations = ["Butte County", "Sacramento"]
number_tweets = 10         #per location
#get old tweets
tweetCriteria_list = []
for location in locations:
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword)\
                                            .setSince(oldest_date)\
                                            .setUntil(newest_date)\
                                            .setNear(location)\
                                            .setWithin('300mi')\
                                            .setMaxTweets(number_tweets)
    tweetCriteria_list.append(tweetCriteria)
#create twitter info for each city
tweet_dict = {}
for criteria, location in zip(tweetCriteria_list, locations):
    tweets = got.manager.TweetManager.getTweets(criteria)
    tweet_dict[location] = tweets
#create df
tweet_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in tweet_dict.items() ]))
tweet_df['tweet_count'] = tweet_df.index
tweet_df = pd.melt(tweet_df, id_vars=["tweet_count"], var_name='City', value_name='got_criteria')
# tweet_df = tweet_df.dropna()
#extract twitter information
get_twitter_info()
tweet_df = tweet_df.drop("got_criteria", 1)
tweet_df.head()

,tweet_count,City,id,tweet_text,date,hashtags,username,mentions,rewtweets,replies,link
0,0,Butte County,1062855646867935232,RISE. #campfire #paradise #california #califor...,2018-11-14 23:51:47+00:00,#campfire #paradise #california #californiafir...,DraperStudios,,0,0,https://twitter.com/DraperStudios/status/10628...
1,1,Butte County,1062853658596069377,The air quality is a reminder of the lost peop...,2018-11-14 23:43:53+00:00,#fire #pray #help,jhowell805,,0,0,https://twitter.com/jhowell805/status/10628536...
2,2,Butte County,1062849282598952960,Showing luv from #tuffcookie4godsownheart We l...,2018-11-14 23:26:30+00:00,#tuffcookie4godsownheart,MizshakurDon,,1,0,https://twitter.com/MizshakurDon/status/106284...
3,3,Butte County,1062848037930835968,"All of the sudden this afternoon, the smoke an...",2018-11-14 23:21:33+00:00,,mmburleson,,0,1,https://twitter.com/mmburleson/status/10628480...
4,4,Butte County,1062846655094050817,Our thoughts and prayers are with those that h...,2018-11-14 23:16:03+00:00,,FederalistAlley,,0,0,https://twitter.com/FederalistAlley/status/106...


In [4]:
# Location from tweet extractor

url = tweet_dict['Butte County'][0].permalink
res = requests.get(url)


soup = BeautifulSoup(res.content, 'lxml')

spans = soup.find_all('span', {'class' : 'permalink-tweet-geo-text'})

spans[0].text.replace("from",'').strip()

200

In [32]:
for span in soup.find_all('span'):
    if span.text == 'Instagram':
        print('find')

In [ ]:
{'class' : 'css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0'}


In [46]:
tweets[22].username

'MembersGetIt'

In [3]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch("CAL_FIRE")\
                                            .setSince("2018-11-01")\
                                           .setUntil("2018-11-30")\
                                            .setNear("Los Angeles, Ca")

tweets = got.manager.TweetManager.getTweets(tweetCriteria)
len(tweets)

7

In [2]:
# Import word vectors into "model."
model = gensim.models.KeyedVectors.load_word2vec_format('../lexvec.enwiki+newscrawl.300d.W.pos.vectors')


In [14]:
keyterms = []
keywords = ['fire', 'smoke', 'wildfire', 'campfire', 'forest', 'evacuate', 'hell']

for word in keywords:
    kt = [x[0] for x in model.most_similar(word, topn = 25)]
    keyterms+= kt
query_list = list(set(keyterms))